In [25]:
import sys
sys.path.append("../")

In [26]:
import numpy
import pandas
import re
import os
from density_calculations import calc_density
from tqdm import tqdm
import matplotlib.pyplot as pyplot
import matplotlib.patches as patches
from random import choice, sample

# These are functions to calculate density with given bins and calculate sensitivity and specificity

### Calculate density for given bins

In [27]:
def calc_density_with_bins(heatmap_benign,
                            heatmap_malignant,
                            bbox,
                            bins
                        ):
    
    
    h, w = heatmap_benign.shape

    #this is used for rescaling the coordinates of boxes
    scale_factor = numpy.array([w, h, w, h], dtype=float)
    
    
    
    x_min, y_min, x_max, y_max = (bbox * scale_factor).astype(int)
    
    #returns counts of elements in each bin
    counts_ben = numpy.unique(numpy.digitize(heatmap_benign[y_min:y_max, x_min:x_max], bins=bins), return_index=True, return_counts=True)[-1]
    counts_mal = numpy.unique(numpy.digitize(heatmap_malignant[y_min:y_max, x_min:x_max], bins=bins), return_index=True, return_counts=True)[-1]

    #dividing the highest count by the sum (total)                         
    density_ben = counts_ben.max() / counts_ben.sum()
    density_mal = counts_mal.max() / counts_mal.sum()
    

    return density_ben, density_mal

In [28]:
def calc_density_with_mean(heatmap_benign,
                            heatmap_malignant,
                            bbox,
                            bins
                        ):
    
    
    h, w = heatmap_benign.shape

    #this is used for rescaling the coordinates of boxes
    scale_factor = numpy.array([w, h, w, h], dtype=float)
    
    
    x_min, y_min, x_max, y_max = (bbox * scale_factor).astype(int)    
    
    
    dens_ben = heatmap_benign[y_min:y_max, x_min:x_max].mean()
    dens_mal = heatmap_malignant[y_min:y_max, x_min:x_max].mean()


    return density_ben, density_mal


### Load saved heatmaps and calculate density, use single thresholds

In [29]:
def load_and_process_with_bins(threshold_benign,
                               threshold_malignant,
                               bins):
  
    if not isinstance(threshold_benign, numpy.ndarray):
        raise TypeError(f"expected threshold_benign to be numpy.ndarra, got {type(threshold_benign)}")

    if not (threshold_benign.ndim == 1):
        raise ValueError(f"expected threshold_benign to have shape [N], but got {threshold_benign.shape}")

    if not numpy.all((0 < threshold_benign) & (threshold_benign < 1)):
        raise ValueError(f"expected threshold_benign values to be in range 0 to 1, but got {threshold_benign}")


    if not isinstance(threshold_malignant, numpy.ndarray):
        raise TypeError(f"expected threshold_malignant to be numpy.ndarra, got {type(threshold_malignant)}")

    if not (threshold_malignant.ndim == 1):
        raise ValueError(f"expected threshold_malignant to have shape [N], but got {threshold_malignant.shape}")

    if not numpy.all((0 < threshold_malignant) & (threshold_malignant < 1)):
        raise ValueError(f"expected threshold_malignant values to be in range 0 to 1, but got {threshold_malignant}")


    if not isinstance(bins, numpy.ndarray):
        raise TypeError(f"expected bins to be type of numpy.ndarray, but got {type(bins)}")

    if not (bins.ndim == 1):
        raise ValueError(f"expected bins to have shape [N], but got {bins.shape}")

    if not numpy.all((bins >=0) & (bins <=1)):
        raise ValueError(f"expected bins elemets to be in [0, 1]")

    if not numpy.all(bins[1:] > bins[:-1]):
        raise ValueError(f"expected bins to be strictly monotonic sequence")

    path = "/home/server/other_projects/breast_cancer/DATA_PATH/temp_files"

    files = os.listdir(path)
    files.sort(key = lambda x: int(re.match(r"image(\d+).+\.npy", x)[1]))

    N = len(files) // 3

    main_matrix = numpy.zeros((3, 3))

    for idx in tqdm(range(N)):
        
        temp = files[3* idx:3*idx + 3]
        temp.sort(key = lambda x: 0 if "bbox" in x else 1 if "ben" in x else 2)
        
        with open(os.path.join(path, temp[0]), 'rb') as f:

            bbox = numpy.load(f)

        with open(os.path.join(path, temp[1]), 'rb') as f:

            heatmap_benign = numpy.load(f)

        with open(os.path.join(path, temp[2]), 'rb') as f:
            
            heatmap_malignant = numpy.load(f)

        #the re.match used to get the annotations from the file names
        annot = re.match(".+_(.+).npy", temp[1])[1]
        
        dens_ben, dens_mal = calc_density_with_bins(heatmap_benign, heatmap_malignant, bins)
        
        
        if annot in ["2", "3"]:
            true_label = 1
        elif annot in ["4a", "4b", "4c", "4", "5", "6"]:
            true_label = 2
        else:
            true_label = 0

        is_benign = dens_ben > threshold_benign
        is_malignant = dens_mal > threshold_malignant
        

        ##DANGER
        if is_benign and is_malignant:
            if dens_ben > dens_mal:
                pred_label = 1
            else:
                pred_label = 2
        elif is_benign:
            pred_label = 1
        elif is_malignant:
            pred_label = 2
        else:
            pred_label = 0
        ##
        
        main_matrix[true_label, pred_label] += 1
        
    specificity = numpy.zeros(3)    
    sensitivity = numpy.zeros(3)
    
    ##specificity
    
    specificity[0] = (main_matrix[1, 1] + main_matrix[2, 2])/\
    (main_matrix[1, 0] + main_matrix[2, 0] + main_matrix[1, 1] + main_matrix[2, 2])
    
    specificity[1] = (main_matrix[0, 0] + main_matrix[2, 2])/\
    (main_matrix[0, 1] + main_matrix[2, 1] + main_matrix[0, 0] + main_matrix[2, 2])
    
    specificity[2] = (main_matrix[0, 0] + main_matrix[1, 1])/\
    (main_matrix[0, 2] + main_matrix[1, 2] + main_matrix[0, 0] + main_matrix[1, 1])
    
    
    ##sensitivity
    
    sensitivity[0] = main_matrix[0, 0]/(main_matrix[0].sum())
    sensitivity[1] = main_matrix[1, 1]/(main_matrix[1].sum())
    sensitivity[2] = main_matrix[2, 2]/(main_matrix[2].sum())
    
    
    #mean accuracy
    
    corrects = main_matrix[0, 0] + main_matrix[1, 1] + main_matrix[2, 2]
    mean_accuracy = corrects / main_matrix.sum()
    
    return main_matrix, specificity, sensitivity, mean_accuracy
    


### Load saved heatmaps and calculate density, use multiple thresholds

In [42]:
def load_and_process_with_bins_multiple(threshold_benign,
                                        threshold_malignant,
                                        bins):
  
    if not isinstance(threshold_benign, numpy.ndarray):
        raise TypeError(f"expected threshold_benign to be numpy.ndarra, got {type(threshold_benign)}")

    if not (threshold_benign.ndim == 1):
        raise ValueError(f"expected threshold_benign to have shape [N], but got {threshold_benign.shape}")

    if not numpy.all((0 < threshold_benign) & (threshold_benign < 1)):
        raise ValueError(f"expected threshold_benign values to be in range 0 to 1, but got {threshold_benign}")


    if not isinstance(threshold_malignant, numpy.ndarray):
        raise TypeError(f"expected threshold_malignant to be numpy.ndarra, got {type(threshold_malignant)}")

    if not (threshold_malignant.ndim == 1):
        raise ValueError(f"expected threshold_malignant to have shape [N], but got {threshold_malignant.shape}")

    if not numpy.all((0 < threshold_malignant) & (threshold_malignant < 1)):
        raise ValueError(f"expected threshold_malignant values to be in range 0 to 1, but got {threshold_malignant}")


    if not isinstance(bins, numpy.ndarray):
        raise TypeError(f"expected bins to be type of numpy.ndarray, but got {type(bins)}")

    if not (bins.ndim == 1):
        raise ValueError(f"expected bins to have shape [N], but got {bins.shape}")

    if not numpy.all((bins >=0) & (bins <=1)):
        raise ValueError(f"expected bins elemets to be in [0, 1]")

    if not numpy.all(bins[1:] > bins[:-1]):
        raise ValueError(f"expected bins to be strictly monotonic sequence")

    path = "../Sample_data/generated_heatmaps"

    files = os.listdir(path)
    files.sort(key = lambda x: int(re.match(r"image(\d+).+\.npy", x)[1]))

    N = len(files) // 3
    
    M = threshold_benign.shape[0] * threshold_malignant.shape[0]
    main_matrix = numpy.zeros((M, 3, 3))

    for idx in tqdm(range(N)):
        
        temp = files[3* idx:3*idx + 3]
        temp.sort(key = lambda x: 0 if "bbox" in x else 1 if "ben" in x else 2)
        
        with open(os.path.join(path, temp[0]), 'rb') as f:

            bbox = numpy.load(f)

        with open(os.path.join(path, temp[1]), 'rb') as f:

            heatmap_benign = numpy.load(f)

        with open(os.path.join(path, temp[2]), 'rb') as f:
            
            heatmap_malignant = numpy.load(f)

        #the re.match used to get the annotations from the file names
        annot = re.match(".+_(.+).npy", temp[1])[1]
        
        dens_ben, dens_mal = calc_density_with_bins(heatmap_benign, heatmap_malignant, bins)
        
        
        if annot in ["2", "3"]:
            true_label = 1
        elif annot in ["4a", "4b", "4c", "4", "5", "6"]:
            true_label = 2
        else:
            true_label = 0

        is_benign = dens_ben > threshold_benign
        is_malignant = dens_mal > threshold_malignant
        
        count = 0
        
        for ben in is_benign:
            for mal in is_malignant:

                ##DANGER
                if ben and mal:
                    if dens_ben > dens_mal:
                        pred_label = 1
                    else:
                        pred_label = 2
                elif ben:
                    pred_label = 1
                elif mal:
                    pred_label = 2
                else:
                    pred_label = 0
                ##

                main_matrix[count, true_label, pred_label] += 1
                count += 1

    specificity = numpy.zeros((M, 3))    
    sensitivity = numpy.zeros((M, 3))
    
    
    ##specificity
    
    specificity[:, 0] = (main_matrix[:, 1, 1] + main_matrix[:, 2, 2])/\
    (main_matrix[:, 1, 0] + main_matrix[:, 2, 0] + main_matrix[:, 1, 1] + main_matrix[:, 2, 2])
    
    specificity[:, 1] = (main_matrix[:, 0, 0] + main_matrix[:, 2, 2])/\
    (main_matrix[:, 0, 1] + main_matrix[:, 2, 1] + main_matrix[:, 0, 0] + main_matrix[:, 2, 2])
    
    specificity[:, 2] = (main_matrix[:, 0, 0] + main_matrix[:, 1, 1])/\
    (main_matrix[:, 0, 2] + main_matrix[:, 1, 2] + main_matrix[:, 0, 0] + main_matrix[:, 1, 1])
    
    
    ##sensitivity
    
    sensitivity[:, 0] = main_matrix[:, 0, 0]/(main_matrix[:, 0].sum(axis=-1))
    sensitivity[:, 1] = main_matrix[:, 1, 1]/(main_matrix[: ,1].sum(axis=-1))
    sensitivity[:, 2] = main_matrix[:, 2, 2]/(main_matrix[:, 2].sum(axis=-1))
    
    
    #mean accuracy
    
    corrects = main_matrix[:, 0, 0] + main_matrix[:, 1, 1] + main_matrix[:, 2, 2]
    mean_accuracy = corrects / main_matrix.sum(axis=(-1, -2))
    
    return main_matrix, specificity, sensitivity, mean_accuracy

### Load saved heatmaps and calculate density, use multiple thresholds, only for [not benign, benign] cases

In [43]:
def load_and_process_with_bins_multiple_only_benign(threshold_benign,
                                        bins):
  
    if not isinstance(threshold_benign, numpy.ndarray):
        raise TypeError(f"expected threshold_benign to be numpy.ndarra, got {type(threshold_benign)}")

    if not (threshold_benign.ndim == 1):
        raise ValueError(f"expected threshold_benign to have shape [N], but got {threshold_benign.shape}")

    if not numpy.all((0 < threshold_benign) & (threshold_benign < 1)):
        raise ValueError(f"expected threshold_benign values to be in range 0 to 1, but got {threshold_benign}")


    if not isinstance(bins, numpy.ndarray):
        raise TypeError(f"expected bins to be type of numpy.ndarray, but got {type(bins)}")

    if not (bins.ndim == 1):
        raise ValueError(f"expected bins to have shape [N], but got {bins.shape}")

    if not numpy.all((bins >=0) & (bins <=1)):
        raise ValueError(f"expected bins elemets to be in [0, 1]")

    if not numpy.all(bins[1:] > bins[:-1]):
        raise ValueError(f"expected bins to be strictly monotonic sequence")

    path = "../Sample_data/generated_heatmaps"

    files = os.listdir(path)
    files.sort(key = lambda x: int(re.match(r"image(\d+).+\.npy", x)[1]))

    N = len(files) // 3
    M = threshold_benign.shape[0]
    main_matrix = numpy.zeros((M, 2, 2))
    
    
    for idx in tqdm(range(N)):
        
        temp = files[3* idx:3*idx + 3]
        temp.sort(key = lambda x: 0 if "bbox" in x else 1 if "ben" in x else 2)
        
        with open(os.path.join(path, temp[0]), 'rb') as f:

            bbox = numpy.load(f)

        with open(os.path.join(path, temp[1]), 'rb') as f:

            heatmap_benign = numpy.load(f)

        with open(os.path.join(path, temp[2]), 'rb') as f:
            
            heatmap_malignant = numpy.load(f)

        #the re.match used to get the annotations from the file names
        annot = re.match(".+_(.+).npy", temp[1])[1]
        
        
        dens_ben, dens_mal = calc_density_with_bins(heatmap_benign, heatmap_malignant, bbox, bins)
        
        
        if annot in ["2", "3"]:
            true_label = 1
        else:
            true_label = 0
            

        is_benign = dens_ben > threshold_benign
            
        main_matrix[ is_benign, true_label, 1] += 1        
        main_matrix[~is_benign, true_label, 0] += 1

    specificity = numpy.zeros((M, 2))    
    sensitivity = numpy.zeros((M, 2))
    
    
    ##specificity
    
    specificity[:, 0] = (main_matrix[:, 1, 1])/\
    (main_matrix[:, 1, 0] + main_matrix[:, 1, 1])
    
    specificity[:, 1] = (main_matrix[:, 0, 0])/\
    (main_matrix[:, 0, 1] + main_matrix[:, 0, 0])

    
    ##sensitivity
    
    sensitivity[:, 0] = main_matrix[:, 0, 0]/(main_matrix[:, 0].sum(axis=-1))
    sensitivity[:, 1] = main_matrix[:, 1, 1]/(main_matrix[: ,1].sum(axis=-1))
    
    #mean accuracy
    
    corrects = main_matrix[:, 0, 0] + main_matrix[:, 1, 1]
    mean_accuracy = corrects / main_matrix.sum(axis=(-1, -2))
    
    return main_matrix, specificity, sensitivity, mean_accuracy

### Load saved heatmaps and calculate density, use multiple thresholds, only for [not benign, benign]. Bins are generated inside the function.

In [60]:
def load_and_process_with_bins_multiple_only_benign_gen_bins(threshold_benign):
  
    if not isinstance(threshold_benign, numpy.ndarray):
        raise TypeError(f"expected threshold_benign to be numpy.ndarra, got {type(threshold_benign)}")

    if not (threshold_benign.ndim == 1):
        raise ValueError(f"expected threshold_benign to have shape [N], but got {threshold_benign.shape}")

    if not numpy.all((0 < threshold_benign) & (threshold_benign < 1)):
        raise ValueError(f"expected threshold_benign values to be in range 0 to 1, but got {threshold_benign}")

    
    path = "../Sample_data/generated_heatmaps"

    files = os.listdir(path)
    files.sort(key = lambda x: int(re.match(r"image(\d+).+\.npy", x)[1]))


    #How many bin combinations
    bins_count = 100
    #How many bins
    len_choices = [5, 6, 7, 8]
    
    
    all_bins = []
    #Generating random bins
    for it in range(bins_count):
        
        N = choice(len_choices)
        all_bins.append(generate_bins(N))
        
    
    N = len(files) // 3
    M = threshold_benign.shape[0]
    main_matrix = numpy.zeros((bins_count, M, 3, 3))
    
    
    for idx in tqdm(range(N)):
        
        temp = files[3* idx:3*idx + 3]
        temp.sort(key = lambda x: 0 if "bbox" in x else 1 if "ben" in x else 2)
        
        with open(os.path.join(path, temp[0]), 'rb') as f:

            bbox = numpy.load(f)

        with open(os.path.join(path, temp[1]), 'rb') as f:

            heatmap_benign = numpy.load(f)


        #the re.match used to get the annotations from the file names
        annot = re.match(".+_(.+).npy", temp[1])[1]
        
        for bin_index, bins in enumerate(all_bins):

            dens_ben, dens_mal = calc_density_with_bins(heatmap_benign, heatmap_benign, bbox, bins)


            if annot in ["2", "3"]:
                true_label = 1
            elif annot in ["4", "4a", "4b", "4c", "5", "6"]:
                true_label = 2
            else:
                true_label = 0


            is_benign = dens_ben > threshold_benign

            main_matrix[bin_index,  is_benign, true_label, 1] += 1        
            main_matrix[bin_index, ~is_benign, true_label, 0] += 1

    specificity = numpy.zeros((bins_count, M, 2))    
    sensitivity = numpy.zeros((bins_count, M, 2))
    
    
    ##specificity
    
    specificity[:, :, 0] = (main_matrix[:, :, 1, 1])/\
    (main_matrix[:, :, 1, 0] + main_matrix[:, :, 1, 1])
    
    specificity[:, :, 1] = (main_matrix[:, :, 0, 0])/\
    (main_matrix[:, :, 0, 1] + main_matrix[:, :, 0, 0])

    
    ##sensitivity
    
    sensitivity[:, :, 0] = main_matrix[:, :, 0, 0]/(main_matrix[:, :, 0].sum(axis=-1))
    sensitivity[:, :, 1] = main_matrix[:, :, 1, 1]/(main_matrix[:, : ,1].sum(axis=-1))
    
    #mean accuracy
    
    corrects = main_matrix[:, :, 0, 0] + main_matrix[:, :, 1, 1]
    mean_accuracy = corrects / main_matrix[..., :2, :2].sum(axis=(-1, -2))
    
    return main_matrix, specificity, sensitivity, mean_accuracy, all_bins

### Generate random bins

In [61]:
def generate_bins(N):

    temp = numpy.linspace(0, 1, 50)
    all_inidices = [i for i in range(1, temp.shape[0]-1)]
    
    inidices = sorted(sample(all_inidices, N - 2))
    
    result = temp[inidices] 
    
    return numpy.concatenate([numpy.array([0]), result, numpy.array([1])])

# Gridsearch over the threshold values, bins are generated inside the function

In [62]:
t_b = numpy.linspace(0.1, 0.99999, 1000)

main_matrix, specificity, sensitivity, mean_accuracy, all_bins = \
load_and_process_with_bins_multiple_only_benign_gen_bins(t_b)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 316/316 [00:03<00:00, 79.13it/s]


# Best mean accuracy value

In [69]:
best_index = numpy.argmax(mean_accuracy)
best_index = numpy.unravel_index(best_index, mean_accuracy.shape)
mean_accuracy[best_index[0], best_index[1]]

0.8120300751879699

# Sensitivity accortind the best index

In [73]:
sensitivity[best_index[0], best_index[1]]

array([0.16666667, 1.        ])

# Specificity accortind the best index

In [74]:
specificity[best_index[0], best_index[1]]

array([1.        , 0.16666667])

# Bins according the best index

In [76]:
all_bins[best_index[0]]

array([0.        , 0.08163265, 0.16326531, 0.26530612, 0.48979592,
       0.59183673, 0.75510204, 1.        ])

# Bins according the best index

In [78]:
t_b[best_index[1]]

0.40450112112112113

# Checking highest benign recall case

In [83]:
indices = numpy.where((sensitivity[..., 1] > 0.60) & (sensitivity[..., 0] > 0.53))

In [84]:
main_matrix[indices[0][-1], indices[1][-1]]

array([[16., 14.,  0.],
       [41., 62.,  0.],
       [96., 87.,  0.]])

In [85]:
mean_accuracy[indices[0][-1], indices[1][-1]]

0.5864661654135338

In [86]:
sensitivity[indices[0][-1], indices[1][-1]]

array([0.53333333, 0.60194175])

In [87]:
all_bins[indices[0][-1]]

array([0.        , 0.06122449, 0.18367347, 0.20408163, 0.36734694,
       0.51020408, 0.87755102, 1.        ])

In [88]:
t_b[indices[1][-1]]

0.7738663863863865